In [1]:
import os, sys
import hashlib

In [2]:
def isWin32Exe(f):
    try:
        with open(f, 'rb') as hnd:
            if hnd.read(2) != b"MZ":
                return False
            hnd.seek(0x3c, 0)

            import struct
            offset = struct.unpack('i', hnd.read(4))[0]
            hnd.seek(offset, 0)
            if hnd.read(4) == b"PE\0\0":
                ## MIKE: for 64 or 32 bit
                machine = struct.unpack("<H", hnd.read(2))[0]
                if machine == 332: # IMAGE_FILE_MACHINE_I386
                    #print(f, "IA-32 (32-bit x86)")
                    return True
                elif machine == 512: #IMAGE_FILE_MACHINE_IA64:
                    #print(f, "IA-64 (Itanium)")
                    pass
                elif machine == 34404: #IMAGE_FILE_MACHINE_AMD64:
                    #print(f, "AMD64 (64-bit x86)")
                    pass
                else:
                    #print(f, "Unknown architecture")
                    pass
                ## end
                return False
            else:
                #print(f, "isMZ, not PE.")
                return False
        
    except:
        #print(f, "Permission denied")
        return False

def md5(f):
    return hashlib.md5(open(f, 'rb').read()).hexdigest()

In [3]:
def iterateFolder(pathToVisit_set, func):
    fileCount = 0
    while len(pathToVisit_set) > 0:
        try:
            curPath = pathToVisit_set.pop()
            curPath = curPath.decode('utf-8') if not isinstance(curPath, str) else curPath # deal with utf8 path
            
            # this might cause except, ususlly dir permission denial
            curPathInfo = next(os.walk(curPath))# if os.path.isdir(curPath) else ([], [] , [curPath])
            
            # sub-dir
            for subdir in curPathInfo[1]:
                pathToVisit_set.add(os.path.join(curPath, subdir))
                
            # for all files in this dir
            for curFile in curPathInfo[2]:
                filePath = os.path.join(curPath, curFile)
                if func(filePath):
                    fileCount += 1
        except:
            # usually permission denial
            pass

    return fileCount

In [4]:
max_level = 3 # For example, C, windows, system = 3
output = "Z:\\save\\" # must use \\
map_dict = dict()

def my_func(f):
    global level, output
    global map_dict
    if isWin32Exe(f):
        tokens = f.split("\\")
        my_level = max_level if len(tokens) - 1 > max_level else len(tokens) - 1
        out_dir = ""
        for t in tokens[0:my_level]:
            out_dir += t+"\\" if t[1] != ":" else t[0]+"\\"
        map_dict[f] = os.path.join(output, out_dir, md5(f))
        return True
    else:
        return False

def _func(f):
    print(f)

In [5]:
pathToVisit_set = set()
pathToVisit_set.add("C:\\ProgramData\\")
iterateFolder(pathToVisit_set, my_func)

163

In [6]:
print(len(map_dict))
for k in map_dict:
    print(k)
    print(map_dict[k])

163
C:\ProgramData\Package Cache\{a9528995-e130-4501-ae19-bbfaddb779cc}\VisualCppBuildTools_Full.exe
Z:\save\C\ProgramData\Package Cache\260f449199cb66be4ddbd5ba5e1e0d98
C:\ProgramData\Package Cache\{f144e08f-9cbe-4f09-9a8c-f2b858b7ee7f}\VC_redist.x64.exe
Z:\save\C\ProgramData\Package Cache\5cb898507fe44cc3a4f184d3fc9b261e
C:\ProgramData\Package Cache\{e2803110-78b3-4664-a479-3611a381656a}\VC_redist.x86.exe
Z:\save\C\ProgramData\Package Cache\24043d621e555c7030908726532654d1
C:\ProgramData\Package Cache\{448652c1-f5f3-4230-98c6-68c10c88b1fb}\vcredist_x64.exe
Z:\save\C\ProgramData\Package Cache\549f50510650f4f485b8cbc251f179d7
C:\ProgramData\Package Cache\{f65db027-aff3-4070-886a-0d87064aabb1}\vcredist_x86.exe
Z:\save\C\ProgramData\Package Cache\2335ab0c0e19c0ef416d07df66fee649
C:\ProgramData\Microsoft\IdentityCRL\INT\ppcrlconfig600.dll
Z:\save\C\ProgramData\Microsoft\69c7c4319ba97a38d0e31191b30519d2
C:\ProgramData\Package Cache\{050d4fc8-5d48-4b8f-8972-47c82c46020f}\vcredist_x64.exe
Z:

In [7]:
import os
import shutil

#for k in map_dict:
#    shutil.copy(k, map_dict[k])